In [1]:
"""
Name: fd_Social_Distancing.ipynb
Authors: Stephan Meighen-Berger, Martina Karl
Example how to use fourth_day with social distancing
"""

'\nName: fd_Social_Distancing.ipynb\nAuthors: Stephan Meighen-Berger, Martina Karl\nExample how to use fourth_day with social distancing\n'

In [2]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm

In [3]:
# Latex text rendering
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

In [4]:
# Adding path to module
sys.path.append("../core/")

In [5]:
# Module imports
from fourth_day import FD
from fd_config import config

In [6]:
# Creating a fourth_day object
fd = FD(org_filter='generous')

In [7]:
# Launching solver
seconds = 10000
pop = 3000
infected=5

In [8]:
# Plotting standards
std_size = 10.
fontsize = 20.
lw=3.
h_length=1.5

In [ ]:
res_store = []
mean_vel = [1., 1.5, 2.]
for i in range(3):
    res_store.append(fd.solve(pop, mean_vel[i], 2., None, seconds=seconds, vel_var=1., dist_var=1., infected=10))

../core\fd_roll_dice_social.py:285: RuntimeWarning: invalid value encountered in double_scalars
  (np.linalg.norm(new_vec) * np.linalg.norm(current_vec))


In [ ]:
# Binning function
def binning(array, s_bin=10):
    return np.array([
        np.sum(array[i*s_bin:(i+1)*s_bin])
        for i in range(int(len(array)/s_bin))
    ])

In [ ]:
# Binning
bin_size = 100
res_binned = np.array([
    binning(np.sum(res_store[i], axis=1), s_bin=bin_size)
    for i in range(len(res_store))
])

In [ ]:
# Custom function
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [ ]:
from scipy.signal import savgol_filter
res_smoothed = np.array([
    savgol_filter(res_binned[i], 31, 3)
    for i in range(len(res_binned))
])

In [ ]:
# Energy distributions
figure, ax = plt.subplots(figsize=(std_size, std_size * 6. / 8.))
colors = ['#fdcc8a',
          '#fc8d59',
          '#d7301f']
for i, _ in enumerate(mean_vel):
    ax.errorbar(
        fd.t[::bin_size], 
        res_binned[i],
        yerr=np.sqrt(res_binned[i]),
        fmt='o',
        markersize=lw, color=colors[i],
        alpha=0.5,
        label=r'$v_{social} = %.1f$' %mean_vel[i]
    )
    ax.plot(
        fd.t[::bin_size], 
        res_smoothed[i],
        lw=lw, color=colors[i],
    )
ax.set_xlim(1e0, 4000)
ax.set_ylim(0., 200)
ax.set_xscale('linear')
ax.set_yscale('linear')
ax.set_xlabel(r'$t$', fontsize=fontsize)
ax.set_ylabel(r'$Infections$', fontsize=fontsize)
ax.tick_params(axis = 'both', which = 'major', labelsize=fontsize, direction='in')
ax.tick_params(axis = 'both', which = 'minor', labelsize=fontsize, direction='in')
plt.grid(True)
h, l = ax.get_legend_handles_labels()
lgd1 = ax.legend(h,l, loc=9, bbox_to_anchor=(0.8, +1.),
                 ncol=1, fontsize=fontsize, handlelength=h_length,
                 fancybox=True, frameon=True)
ax.add_artist(lgd1)
plt.show()

In [ ]:
# figure.savefig("C:\\Users\\steph\\OneDrive\\University TUM\\PhD\\Plots\\Biolumi\\MC_Social_Rate_pop_2000_infected_5.png",
#                bbox_inches='tight')

In [ ]:
"""
# Animation stack
# This requires "save population" in config to be set to true
fontsize = 15.
import matplotlib.colors as colors
# The frames are created in a loop
cmap = plt.cm.viridis
norm = colors.Normalize(vmin=0., vmax=1.)
for i, _ in tqdm(enumerate(fd.t)):
    figure = plt.figure(figsize=(std_size, std_size * 6. / 8.))
    ax = figure.add_subplot(111)
    ax.scatter(fd.mc_run.distribution[i][:,0], fd.mc_run.distribution[i][:,1],
               color=cmap(norm(fd.mc_run.distribution[i][:,-1])),
               s=50.
               )
    ax.title=ax.set_title('Population t = %.f' %fd.t[i], fontsize=fontsize)
    ax.set_xlim(-fd.world.bounding_box/2, fd.world.bounding_box/2)
    ax.set_ylim(-fd.world.bounding_box/2, fd.world.bounding_box/2)
    ax.set_xlabel(r'$X$', fontsize=fontsize)
    ax.set_ylabel(r'$Y$', fontsize=fontsize)
    ax.tick_params(axis = 'both', which = 'major', labelsize=fontsize)
    ax.tick_params(axis = 'both', which = 'minor', labelsize=fontsize)
    figure.savefig('C:\\Users\\steph\\OneDrive\\University TUM\\PhD\\Plots\\Biolumi\\Frames\\frame_%d.png' %i)
    plt.close(figure)
import imageio
images = []
filenames = np.array([
    'C:\\Users\\steph\\OneDrive\\University TUM\\PhD\\Plots\\Biolumi\\Frames\\frame_%d.png' %i
    for i in range(len(fd.t))
])
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('C:\\Users\\steph\\OneDrive\\University TUM\\PhD\\Plots\\Biolumi\\Social_Distancing.gif',
                images, duration=0.1)
"""